In [ ]:
import requests
import json
import os
from math import ceil

defaultPath = os.getcwd()

def getJson(url):
    getJsonRequests = requests.get(url)
    getJsonLoads = json.loads(getJsonRequests.text)
    return getJsonLoads

def pictureSave(name,url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.114 Safari/537.36','Cookie': 'AspxAutoDetectCookieSupport=1',}
    with open(name.format(),'wb') as file:
        file.write(requests.get(url,headers = headers).content)

inputString = input("请输入用户ID:")

idPage = 'http://api.topitme.com/?appVersion=508&device=ios&build=4.3.13&ipad=NO&ch=AppStore&openudid=8760362a232719eb2c1bd3d745bcc1e20bf310ad&screen=1242x2208&id=' + inputString + '&method=user.get&offset=0&limit=30'
idJson = getJson(idPage)

uid = idJson['info']['sbj']['id']
originalAlbumUrlLeft = 'http://api.topitme.com/?appVersion=508&device=ios&build=4.3.13&ipad=YES&ch=AppStore&openudid=d8793876f280c8d5808d0faaf0dc8b8d96d679f4&screen=1536x2048&id='
originalAlbumUrlMiddle = '&data_ref=id%253D3451964%2526method%253Duser.get%2526offset%253D0%2526limit%253D30&type=0&offset='
originalAlbumUrlRight = '&limit=30'

originalAlbumUrl = idJson['info']['category'][0]['more']['next']
originalAlbumNum = idJson['info']['category'][0]['more']['num']
originalAlbumUrlList = []
originalAlbumItemUrlList = []
originalAlbumName = []
fnum = 0

urlLeft = 'http://api.topitme.com/?appVersion=508&device=ios&build=4.3.13&ipad=YES&ch=AppStore&openudid=d8793876f280c8d5808d0faaf0dc8b8d96d679f4&screen=1536x2048&id='
urlRight = '&method=item.getIcon&type=o'

os.makedirs('原创的精选集')
os.chdir('原创的精选集')

for i in range(ceil(int(originalAlbumNum)/30)):
    url = originalAlbumUrlLeft + str(uid) + '&method=user.getAlbums' + originalAlbumUrlMiddle + str(i * 30) + originalAlbumUrlRight
    originalAlbumUrlList.append(url)

for i in originalAlbumUrlList:
    originalAlbumJsonLoad = getJson(i)
    for j in range(len(originalAlbumJsonLoad['item'])):
        aid = originalAlbumJsonLoad['item'][j]['id']
        anum = originalAlbumJsonLoad['item'][j]['onum']
        aname = originalAlbumJsonLoad['item'][j]['name']
        originalAlbumName.append(aname)
        originalAlbumItemUrlList.append([])
        for k in range(ceil(int(anum)/30)):
            aurl = originalAlbumUrlLeft + str(aid) + '&method=album.get' + originalAlbumUrlMiddle + str(k * 30) + originalAlbumUrlRight
            originalAlbumItemUrlList[fnum].append(aurl)
        fnum += 1

for i in range(len(originalAlbumItemUrlList)):
    os.makedirs(originalAlbumName[i])
    os.chdir(originalAlbumName[i])
    for j in originalAlbumItemUrlList[i]:
        pictureJsonLoad = getJson(j)
        for k in pictureJsonLoad['item']:
            pictureId = k['id']
            pictureContentedUrl = urlLeft + pictureId + urlRight
            pictureUrl = getJson(pictureContentedUrl)['item'][0]['icon']['url']
            pictureSave(pictureUrl.split('/')[-1],pictureUrl)
    os.chdir('原创的精选集')
os.chdir(defaultPath)
